In [1]:
!nvidia-smi
import torch
import os
from ultralytics.models.yolo.detect.train import DetectionTrainer
from ultralytics import YOLO
from mask_attack.utils.dataset import CustomDataset
# os.chdir(path='E://bmx/YoloCAM')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args = dict(model="../models/yolov8-gtsrb.pt", data="./data.yaml")
trainer = DetectionTrainer(overrides=args)
trainer.setup_model()
trainer.set_model_attributes()
trainer.model.to(device=device)
trainer.model.eval()
detector = YOLO(model="../models/yolov8-gtsrb.pt")
detector.to(device=device, dtype=torch.float32).eval()

Thu May 15 19:36:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.94                 Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090      WDDM  |   00000000:01:00.0  On |                  Off |
| 61%   52C    P2            158W /  450W |    8786MiB /  24564MiB |     44%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [2]:
from analyzer.utils import analysis_all_layer_cross_classes
# key_filters = analysis_all_layer_cross_classes(trainer)
key_filters = {
    'backbone_c2f1.cv1': [10, 13, 16],
    'backbone_c2f1.cv2': [27],
    'backbone_c2f1.m.0.cv1': [],
    'backbone_c2f1.m.0.cv2': [],
    'backbone_c2f2.cv1': [7, 18, 21],
    'backbone_c2f2.cv2': [21, 38, 45],
    'backbone_c2f2.m.0.cv1': [6, 9, 12, 28, 29, 31],
    'backbone_c2f2.m.0.cv2': [6, 7, 24],
    'backbone_c2f2.m.1.cv1': [7],
    'backbone_c2f2.m.1.cv2': [],
    'backbone_c2f3.cv1': [12, 20, 21, 22, 39, 58, 61],
    'backbone_c2f3.cv2': [0, 41, 96],
    'backbone_c2f3.m.0.cv1': [28],
    'backbone_c2f3.m.0.cv2.conv': [51],
    'backbone_c2f3.m.1.cv1': [],
    'backbone_c2f3.m.1.cv2': [],
    'backbone_c2f4.cv1': [48],
    'backbone_c2f4.cv2': [91, 122],
    'backbone_c2f4.m.0.cv1': [73, 84],
    'backbone_c2f4.m.0.cv2': [120],
    'neck_c2f1.cv1': [25],
    'neck_c2f1.cv2': [],
    'neck_c2f1.m.0.cv1': [40, 59],
    'neck_c2f1.m.0.cv2': [],
    'neck_c2f2.cv1': [1, 52],
    'neck_c2f2.cv2': [],
    'neck_c2f2.m.0.cv1': [24],
    'neck_c2f2.m.0.cv2': [],
    'neck_c2f3.cv1': [7, 114],
    'neck_c2f3.cv2': [44],
    'neck_c2f3.m.0.cv1': [],
    'neck_c2f3.m.0.cv2': [47, 59],
    'neck_c2f4.cv1': [],
    'neck_c2f4.cv2': [228],
    'neck_c2f4.m.0.cv1': [],
    'neck_c2f4.m.0.cv2': [10, 126]
}

In [ ]:
from mask_attack.utils.attacker import Attacker
origin_dataset = CustomDataset(images_dir_path = "../datasets/gtsrb_origin/0/images", 
                              labels_dir_path = "../datasets/gtsrb_origin/0/labels",
                              image_width=640,
                              image_height=640)
attacker = Attacker(trainer=trainer, dataset=origin_dataset, batch_size=1)
sample = origin_dataset[0]
attacker.single_filter_attack(sample=sample, lr=0.001, epsilon=0.03, num_iter=100, key_filters=key_filters)

ValueError: expected 4D input (got 3D input)